# Лекция: **Квантизация нейронных сетей**

## **Введение:**
Квантизация — ключевой метод оптимизации нейросетей для работы на устройствах с ограниченными ресурсами. Она позволяет уменьшить размер моделей, ускорить вычисления и снизить энергопотребление. В лекции разберём теоретические основы и практические аспекты применения квантизации.

## **Цель работы:**
Понять принципы квантизации, научиться применять её к весам и активациям моделей, оценить компромиссы между точностью и производительностью.

## **Теоретическая часть**
### 1. Основные понятия
**Квантизация** — преобразование данных из формата с высокой точностью (float32) в компактный (int8). Аналогична дискретизации аналогового сигнала.

**Преимущества:**
- Уменьшение размера модели в 4 раза (float32 → int8);
- Ускорение вычислений за счёт оптимизированных операций с целыми числами;
- Экономия энергии для мобильных устройств.

**Типы квантизации:**
- **Аффинная (несимметричная):**
  - Формулы:
    - Масштаб: $S = \frac{r_{\text{max}} - r_{\text{min}}}{q_{\text{max}} - q_{\text{min}}}$
    - Zero-Point: $Z = \left\lfloor q_{\text{min}} - \frac{r_{\text{min}}}{S} \right\rceil$
  - Подходит для асимметричных данных (например, выход ReLU).

- **Симметричная:**
  - Формула масштаба: $S = \frac{|r|_{\text{max}}}{2^{N-1} - 1}$
  - Нулевая точка всегда равна 0.

### 2. Гранулярность
Уровень детализации квантизации:
- **По тензору:** одна константа масштаба на весь тензор.
- **По строкам/столбцам:** отдельные константы для строк/столбцов (повышает точность).
- **По блокам:** например, блоки 64x64 элементов (оптимально для борьбы с выбросами).

### 3. Типы данных
- **INT8:** 8-битные целые (−128 до 127). Основной формат для инференса.
- **BFLOAT16:** 16-битные числа с экспонентой float32. Используется в обучении.
- **NF4:** 4-битный формат для квантизации нормально распределённых весов.

### 4. Проблема выбросов
Выбросы в активациях (значения >> средних) нарушают точность квантизации. Решения:
- **LLM.Int8:** Разделение матриц на части с выбросами и без.
- **SmoothQuant:** Перенос выбросов из активаций в веса.
---
### 5. Математические основы
#### 5.1 Формализация задачи
Для тензора весов $W \in \mathbb{R}^{n \times m}$ найти квантизованное представление $\hat{W} \in \mathbb{Z}^{n \times m}$ и масштаб $S \in \mathbb{R}$, чтобы:
$$
\min_{S, \hat{W}} ||W - S \cdot \hat{W}||_F^2
$$
где $||\cdot||_F$ — норма Фробениуса.

#### 5.2 Кривые искажения
Сравнение ошибок при разных методах квантизации:
```
        равномерная       │       нелинейная
    ●●●●●●●●●●●●●●        │    ▲
    ●              ●       │    ●●●●
    ●              ●       │   ●    ●
━━━━●━━━━━━━━━━━━━━●━━━━   ━━━━●────●━━━
```
Нелинейные методы лучше сохраняют важные диапазоны значений.

### 6. Уровни гранулярности
#### 6.1 Анализ эффективности
| Гранулярность   | Точность (TOP-1) | Скорость (FPS) |
|-----------------|------------------|----------------|
| Тензорная       | 72.3%            | 145            |
| По каналам      | 75.1%            | 132            |
| Блочная 64x64   | 76.8%            | 121            |

#### 6.2 Практический пример
Для матрицы активаций $A \in \mathbb{R}^{3x3}$:
$$
\begin{bmatrix}
1.2 & 25.1 & 3.4 \\
0.5 & 18.6 & 2.9 \\
1.8 & 22.3 & 4.1
\end{bmatrix}
$$
При блочной квантизации 2x2:
1. Разбиваем матрицу на блоки
2. Вычисляем масштаб для каждого блока
3. Кодируем отдельно

### 7. Аппаратная поддержка
#### 7.1 Сравнение архитектур
- **CPU:** AVX-512 VNNI (INT8)
- **GPU:** Tensor Cores (FP8)
- **NPU:** Аппаратные акселераторы для 4-битных операций

#### 7.2 Энергоэффективность
$$
E = \frac{\text{TOPS}}{\text{Вт}} \times \text{Битовая эффективность}
$$
Где битовая эффективность вычисляется как отношение точности квантизованной модели к исходной.

### 8. Продвинутые методы
#### 8.1 Адаптивная квантизация
Динамическая регулировка битности в зависимости от важности слоёв:
```python
def adaptive_quant(layer):
    sensitivity = gradient_norm(layer)
    bits = 8 - 4 * sigmoid(sensitivity)
    return quantize(layer, bits=round(bits))
```

#### 8.2 Гибридные подходы
Комбинация разных типов данных в одной модели:
- Веса: 4-бит NF4
- Активации: 8-бит FP8
- Градиенты: BF16

### 9. Метрики качества
Для оценки эффективности квантизации используют:
1. **PSNR** (пиковое отношение сигнал/шум):
$$
\text{PSNR} = 20 \log_{10}\left(\frac{\text{MAX}_I}{\sqrt{\text{MSE}}}\right)
$$
2. **EMD** (Earth Mover's Distance) для распределений
3. **Perplexity** для языковых моделей

### 10. Кейс-стади: MobileNet v3
| Версия       | Точность | Размер  | Энергопотребление |
|--------------|----------|---------|--------------------|
| FP32         | 75.2%    | 12 MB   | 3.2 Дж/кадр        |
| INT8         | 74.8%    | 3 MB    | 0.9 Дж/кадр        |
| Смешанная    | 75.1%    | 4.5 MB  | 1.1 Дж/кадр        |

### 11. Практические применения квантизации в индустрии
**Кейс 1: Голосовые помощники**  
Модель распознавания ключевого слова "Алиса" в Яндекс.Станции:
- Исходный размер: 15 МБ (FP32)
- После квантизации: 4 МБ (INT8)
- Задержка снижена с 120 мс до 35 мс

**Кейс 2: Автономные автомобили**  
Сеть семантической сегментации NVIDIA DRIVE:
```python
# Псевдокод оптимизации
model = load_model('segnet.fp32')
quant_config = {
    'conv1': {'bits': 8, 'granularity': 'per_channel'},
    'conv5': {'bits': 4, 'method': 'NF4'}
}
quantize_model(model, quant_config)
```

### 12. Сравнение алгоритмов квантизации
| Метод          | Битность | Точность | Поддержка HW | Сложность |
|----------------|----------|----------|--------------|-----------|
| Post-Training  | 8-bit    | ███▌     | Все CPU      | Низкая    |
| QAT            | 4-8 bit  | ████▋    | GPU/TPU      | Средняя   |
| GPTQ           | 3-4 bit  | ██▌      | GPU          | Высокая   |
| SmoothQuant    | 8-bit    | ████▊    | NVIDIA       | Средняя   |

### 13. Влияние на архитектуры нейросетей
**Сверточные сети (CNN):**  
- Чувствительны к квантизации первых слоев
- Решение: гибридная квантизация
$$
\mathcal{L}_{quant} = \alpha||W - \hat{W}||_2 + \beta\mathcal{L}_{task}
$$

**Рекуррентные сети (RNN):**  
Проблема накопления ошибки в скрытых состояниях:
```
Ошибка шага t+1 = f(Ошибка шага t, ε_quant)
```

### 14. Реализация в популярных фреймворках
**PyTorch Dynamic Quantization:**
```python
import torch.quantization

model = torchvision.models.mobilenet_v3_small(pretrained=True)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(quantized_model.state_dict(), 'quantized.pth')
```

**TensorFlow Lite:**
```python
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
```

### 15. Будущие тенденции
1. **Нейроморфные вычисления:** Аналоговая квантизация в мемристорах
2. **Квантовое квантование:** Использование кубитов для сжатия моделей
3. **Биологически вдохновленные методы:** Имитация синаптической пластичности

## **Заключение**
К 2026 году 80% edge-устройств будут использовать квантизованные модели. Ключевые направления:
- Автоматический выбор битности (AutoQ)
- Аппаратно-программная коктейльная оптимизация
- Квантизация с гарантированной точностью

```mermaid
graph LR
A[FP32 Модель] --> B{Выбор метода}
B -->|Мобильные| C[Post-Training]
B -->|Серверные| D[GPTQ]
B -->|Гибридные| E[SmoothQuant]
```


## **Практическая часть**
### Пример 1: Квантизация весов в PyTorch
Квантование предобученной модели ResNet-18.

In [ ]:
import torch
import torchvision.models as models

# Загрузка модели
model = models.resnet18(pretrained=True)
model.eval()

# Настройка квантизации
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Подготовка и преобразование модели
quantized_model = torch.quantization.prepare(model, inplace=False)
quantized_model = torch.quantization.convert(quantized_model)

# Проверка типа весов первого слоя
print(quantized_model.conv1.weight().dtype)  # Вывод: torch.qint8

**Пояснение:**
- `qconfig` задаёт конфигурацию квантизации для CPU.
- `prepare` встраивает наблюдатели для сбора статистики активаций.
- `convert` заменяет слои на квантизованные версии.

### Пример 2: Quantize-Aware Training в TensorFlow
Эмуляция квантизации во время обучения.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow_model_optimization.quantization.keras import quantize_model

# Создание модели
model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dense(10, activation='softmax')
])

# Обёртка для Quantize-Aware Training
qat_model = quantize_model(model)
qat_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Обучение (пример с MNIST)
# qat_model.fit(x_train, y_train, epochs=5)

**Пояснение:**
- `quantize_model` добавляет псевдоквантизационные узлы в слои.
- Во время обратного прохода градиенты вычисляются для исходных весов.

### Пример 3: Реализация SmoothQuant
Масштабирование активаций и весов для борьбы с выбросами.

In [ ]:
import numpy as np

def smooth_quant(activations, weights, alpha=0.5):
    # Вычисление масштабов
    S_act = np.max(np.abs(activations), axis=0)
    S_weights = np.max(np.abs(weights), axis=1)
    S = (S_act ** alpha) / (S_weights ** (1 - alpha))

    # Масштабирование
    activations_scaled = activations / S
    weights_scaled = weights * S[:, np.newaxis]
    return activations_scaled, weights_scaled

# Пример данных
activations = np.random.randn(100, 256)  # Активации
weights = np.random.randn(256, 512)      # Веса слоя
aq, wq = smooth_quant(activations, weights)

**Пояснение:**
- `alpha=0.5` балансирует масштабирование между активациями и весами.
- После масштабирования выбросы в активациях уменьшаются.

## **Заключение**
**Преимущества квантизации:**
- Уменьшение размера модели до 4 раз.
- Ускорение инференса на 2-4x.

**Недостатки:**
- Потеря точности (0.5-2% для INT8).
- Зависимость от аппаратной поддержки (INT8 на GPU vs CPU).

**Рекомендации:**
- Для мобильных устройств: Post-Training Quantization.
- Для LLM: Использовать GPTQ или SmoothQuant.



- [Habr](https://habr.com/ru/companies/yandex/articles/800945/)
- [Методы оптимизации нейронных сетей](https://habr.com/ru/articles/318970/)
- [ServerFlow](https://serverflow.ru/blog/stati/kvantizatsiya-ii-chto-eto-takoe-i-dlya-chego-nuzhno/)